# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 10:05AM,247247,12,U1035,"Uniderm USA, Inc",Released
1,Sep 26 2022 10:05AM,247247,12,U1036,"Uniderm USA, Inc",Released
2,Sep 26 2022 10:05AM,247247,12,U1037,"Uniderm USA, Inc",Released
3,Sep 26 2022 10:05AM,247247,12,U1038,"Uniderm USA, Inc",Released
4,Sep 26 2022 10:05AM,247247,12,U1039,"Uniderm USA, Inc",Released
5,Sep 26 2022 10:05AM,247247,12,U1040,"Uniderm USA, Inc",Released
6,Sep 26 2022 10:05AM,247247,12,U1041,"Uniderm USA, Inc",Released
7,Sep 26 2022 10:05AM,247247,12,U1042,"Uniderm USA, Inc",Released
8,Sep 26 2022 10:05AM,247247,12,U1043,"Uniderm USA, Inc",Released
9,Sep 26 2022 10:03AM,247229,10,0086006191,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,247240,Executing,1
16,247241,Released,13
17,247242,Released,6
18,247245,Released,1
19,247247,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247240,1.0,NaN
247241,NaN,13.0
247242,NaN,6.0
247245,NaN,1.0
247247,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247197,0.0,1.0
247199,0.0,1.0
247215,0.0,1.0
247217,0.0,1.0
247219,0.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247197,0,1
247199,0,1
247215,0,1
247217,0,1
247219,0,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247197,0,1
1,247199,0,1
2,247215,0,1
3,247217,0,1
4,247219,0,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247197,,1
1,247199,,1
2,247215,,1
3,247217,,1
4,247219,,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 10:05AM,247247,12,"Uniderm USA, Inc"
9,Sep 26 2022 10:03AM,247229,10,ISDIN Corporation
54,Sep 26 2022 9:58AM,247245,10,Bio-PRF
55,Sep 26 2022 9:56AM,247199,21,"NBTY Global, Inc."
56,Sep 26 2022 9:54AM,247242,10,"Methapharm, Inc."
62,Sep 26 2022 9:54AM,247197,21,"NBTY Global, Inc."
63,Sep 26 2022 9:53AM,247241,10,"Methapharm, Inc."
76,Sep 26 2022 9:49AM,247227,20,"ACI Healthcare USA, Inc."
77,Sep 26 2022 9:46AM,247240,10,"Goodwin Biotechnology, Inc."
78,Sep 26 2022 9:37AM,247239,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 26 2022 10:05AM,247247,12,"Uniderm USA, Inc",,9
1,Sep 26 2022 10:03AM,247229,10,ISDIN Corporation,,45
2,Sep 26 2022 9:58AM,247245,10,Bio-PRF,,1
3,Sep 26 2022 9:56AM,247199,21,"NBTY Global, Inc.",,1
4,Sep 26 2022 9:54AM,247242,10,"Methapharm, Inc.",,6
5,Sep 26 2022 9:54AM,247197,21,"NBTY Global, Inc.",,1
6,Sep 26 2022 9:53AM,247241,10,"Methapharm, Inc.",,13
7,Sep 26 2022 9:49AM,247227,20,"ACI Healthcare USA, Inc.",,1
8,Sep 26 2022 9:46AM,247240,10,"Goodwin Biotechnology, Inc.",1,
9,Sep 26 2022 9:37AM,247239,10,Bio-PRF,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 10:05AM,247247,12,"Uniderm USA, Inc",9,
1,Sep 26 2022 10:03AM,247229,10,ISDIN Corporation,45,
2,Sep 26 2022 9:58AM,247245,10,Bio-PRF,1,
3,Sep 26 2022 9:56AM,247199,21,"NBTY Global, Inc.",1,
4,Sep 26 2022 9:54AM,247242,10,"Methapharm, Inc.",6,
5,Sep 26 2022 9:54AM,247197,21,"NBTY Global, Inc.",1,
6,Sep 26 2022 9:53AM,247241,10,"Methapharm, Inc.",13,
7,Sep 26 2022 9:49AM,247227,20,"ACI Healthcare USA, Inc.",1,
8,Sep 26 2022 9:46AM,247240,10,"Goodwin Biotechnology, Inc.",,1
9,Sep 26 2022 9:37AM,247239,10,Bio-PRF,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 10:05AM,247247,12,"Uniderm USA, Inc",9,
1,Sep 26 2022 10:03AM,247229,10,ISDIN Corporation,45,
2,Sep 26 2022 9:58AM,247245,10,Bio-PRF,1,
3,Sep 26 2022 9:56AM,247199,21,"NBTY Global, Inc.",1,
4,Sep 26 2022 9:54AM,247242,10,"Methapharm, Inc.",6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 10:05AM,247247,12,"Uniderm USA, Inc",9.0,NaN
1,Sep 26 2022 10:03AM,247229,10,ISDIN Corporation,45.0,NaN
2,Sep 26 2022 9:58AM,247245,10,Bio-PRF,1.0,NaN
3,Sep 26 2022 9:56AM,247199,21,"NBTY Global, Inc.",1.0,NaN
4,Sep 26 2022 9:54AM,247242,10,"Methapharm, Inc.",6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 10:05AM,247247,12,"Uniderm USA, Inc",9.0,0.0
1,Sep 26 2022 10:03AM,247229,10,ISDIN Corporation,45.0,0.0
2,Sep 26 2022 9:58AM,247245,10,Bio-PRF,1.0,0.0
3,Sep 26 2022 9:56AM,247199,21,"NBTY Global, Inc.",1.0,0.0
4,Sep 26 2022 9:54AM,247242,10,"Methapharm, Inc.",6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3708450,199.0,1.0
12,247247,9.0,0.0
16,247238,1.0,0.0
20,247227,1.0,0.0
21,494396,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3708450,199.0,1.0
1,12,247247,9.0,0.0
2,16,247238,1.0,0.0
3,20,247227,1.0,0.0
4,21,494396,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,199.0,1.0
1,12,9.0,0.0
2,16,1.0,0.0
3,20,1.0,0.0
4,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,199.0
1,12,Released,9.0
2,16,Released,1.0
3,20,Released,1.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,20,21
Status,,,,,
Executing,1.0,0.0,0.0,0.0,0.0
Released,199.0,9.0,1.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,20,21
0,Executing,1.0,0.0,0.0,0.0,0.0
1,Released,199.0,9.0,1.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,20,21
0,Executing,1.0,0.0,0.0,0.0,0.0
1,Released,199.0,9.0,1.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()